## Set Up

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mkdir dataset

In [ ]:
cd /content/drive/MyDrive/RotNet

In [ ]:
!pip install wget

In [ ]:
from __future__ import print_function

import os
import sys

from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau
from keras.applications.resnet50 import ResNet50
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Model
from keras.layers import Dense, Flatten
from keras.optimizers import SGD

#sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
from utils import angle_error, RotNetDataGenerator

In [ ]:
import os
import wget
import zipfile

## Download and Split Dataset

In [ ]:
def download(output_dir):
    for i in range(4):
        filename = 'part{}.zip'.format(i + 1)
        url = r'http://www.cs.ucf.edu/~aroshan/index_files/Dataset_PitOrlManh/zipped images/' + filename
        print('Downloading', url)
        filepath = wget.download(url, out=os.path.join(output_dir))

        print('\nExtracting', filename)
        with zipfile.ZipFile(filepath, 'r') as z:
            z.extractall(output_dir)
        os.remove(filepath)

In [ ]:
def get_filenames(path):


    image_paths = []
    for filename in os.listdir(path):
        view_id = filename.split('_')[1].split('.')[0]
        # ignore images with markers (0) and upward views (5)
        if not(view_id == '0' or view_id == '5'):
            image_paths.append(os.path.join(path, filename))

    # 90% train images and 10% test images
    n_train_samples = int(len(image_paths) * 0.9)
    train_filenames = image_paths[:n_train_samples]
    test_filenames = image_paths[n_train_samples:]

    return train_filenames, test_filenames

In [ ]:
download('/content/dataset')


Extracting part1.zip

Extracting part2.zip

Extracting part3.zip

Extracting part4.zip


because the dataset is less we use the street view dataset instead. Dataset :
1. Train : 90% street view + 90% barcode
2. Test : 10% street view + 10% barcode

In [ ]:
#data streetview
data_path = '/content/drive/MyDrive/RotNet/dataset'
train_filenames1, test_filenames1 = get_filenames(data_path)

In [ ]:
#data barcode
data_path = '/content/dataset'
train_filenames2, test_filenames2 = get_filenames(data_path)

In [ ]:
#combine data streetview dan barcode
from distutils.dir_util import copy_tree
copy_tree("/content/dataset", "/content/drive/MyDrive/RotNet/dataset")

In [ ]:
train_filenames = train_filenames2 + train_filenames1
test_filenames = test_filenames2 + test_filenames1
print(len(train_filenames), 'train samples')
print(len(test_filenames), 'test samples')

16601 train samples
1845 test samples


## Transfer Learning

In [ ]:
model_name = 'rotnet_barcode_view_resnet50'

# number of classes
nb_classes = 360
# input image shape
input_shape = (224, 224, 3)

In [ ]:
# load base model
base_model = ResNet50(weights='imagenet', include_top=False,
                      input_shape=input_shape)

# append classification layer
x = base_model.output
x = Flatten()(x)
final_output = Dense(nb_classes, activation='softmax', name='fc360')(x)

# create the new model
model = Model(inputs=base_model.input, outputs=final_output)

model.summary()

94773248/94765736 [==============================] - 2s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_____________________________

In [ ]:
# model compilation
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=0.01, momentum=0.9),
              metrics=[angle_error])

In [ ]:
# training parameters
batch_size = 32
nb_epoch = 50

output_folder = 'models'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# callbacks
monitor = 'val_angle_error'
checkpointer = ModelCheckpoint(
    filepath=os.path.join(output_folder, model_name + '.hdf5'),
    monitor=monitor,
    save_best_only=True
)
reduce_lr = ReduceLROnPlateau(monitor=monitor, patience=3)
early_stopping = EarlyStopping(monitor=monitor, patience=5)
tensorboard = TensorBoard()

## Train

In [ ]:
# training loop
model.fit_generator(
    RotNetDataGenerator(
        train_filenames,
        input_shape=input_shape,
        batch_size=batch_size,
        preprocess_func=preprocess_input,
        crop_center=True,
        crop_largest_rect=True,
        shuffle=True
    ),
    steps_per_epoch=len(train_filenames) / batch_size,
    epochs=nb_epoch,
    validation_data=RotNetDataGenerator(
        test_filenames,
        input_shape=input_shape,
        batch_size=batch_size,
        preprocess_func=preprocess_input,
        crop_center=True,
        crop_largest_rect=True
    ),
    validation_steps=len(test_filenames) / batch_size,
    callbacks=[checkpointer, reduce_lr, early_stopping, tensorboard],
    workers=10
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
518/518 [==============================] - 725s 1s/step - loss: 8.0651 - angle_error: 63.8772 - val_loss: 5.6489 - val_angle_error: 38.0718
Epoch 2/50
518/518 [==============================] - 687s 1s/step - loss: 4.8934 - angle_error: 31.2301 - val_loss: 4.6837 - val_angle_error: 27.0147
Epoch 3/50
518/518 [==============================] - 690s 1s/step - loss: 4.6041 - angle_error: 24.7198 - val_loss: 4.5067 - val_angle_error: 24.9202
Epoch 4/50
518/518 [==============================] - 689s 1s/step - loss: 4.4107 - angle_error: 21.5815 - val_loss: 4.3206 - val_angle_error: 19.6761
Epoch 5/50
518/518 [==============================] - 689s 1s/step - loss: 4.2540 - angle_error: 18.5707 - val_loss: 4.1493 - val_angle_error: 18.4565
Epoch 6/50
518/518 [==============================] - 688s 1s/step - loss: 4.1427 - angle_error: 17.3395 - val_loss: 4.0896 - val_angle_error: 17.6382
Epoch 7/50
518/518 [==============================] - 690s 1s/step - loss: 4.0372 - angle_erro

In [ ]:
#move model to drive
from distutils.dir_util import copy_tree
copy_tree("/content/RotNet/models", "/content/drive/MyDrive/RotNet/models")